In [0]:
%fs ls abfss://datalake@datalakegen2storagelele.dfs.core.windows.net

path,name,size,modificationTime
abfss://datalake@datalakegen2storagelele.dfs.core.windows.net/example/,example/,0,1697901489000


In [0]:
%python
configs = {
    "fs.azure.account.auth.type": "CustomAccessToken",
    "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}
dbutils.fs.mount(
    source = "abfss://datalake@datalakegen2storagelele.dfs.core.windows.net/",
    mount_point = "/mnt/datalake",
    extra_configs = configs
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4367303151985346>, line 5
      1 configs = {
      2     "fs.azure.account.auth.type": "CustomAccessToken",
      3     "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
      4 }
----> 5 dbutils.fs.mount(
      6     source = "abfss://datalake@datalakegen2storagelele.dfs.core.windows.net/",
      7     mount_point = "/mnt/datalake",
      8     extra_configs = configs
      9 )

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o404.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement f

In [0]:
%python
spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")


'com.databricks.backend.daemon.data.client.adl.AdlGen2CredentialContextTokenProvider'

In [0]:
%fs ls /mnt/datalake/example

path,name,size,modificationTime
dbfs:/mnt/datalake/example/flipkart_fashion_products_dataset.json,flipkart_fashion_products_dataset.json,82720262,1697909569000
dbfs:/mnt/datalake/example/test.json,test.json,476,1697983136000


In [0]:
%python
df = spark.read.json(sc.wholeTextFiles("/mnt/datalake/example/flipkart_fashion_products_dataset.json").values())
df.show()


+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
|                 _id|actual_price|average_rating|brand|            category|          crawled_at|         description|discount|              images|out_of_stock|             pid|     product_details|           seller|selling_price|sub_category|               title|                 url|
+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
|fa8e22d6-c0b6-522...|       2,999|           3.9| York|Clothing and Acce...|02/10/2021, 20:11:51|Yorker trackpants...| 69% off|[https:/

## Calculate Top 10 Brands with the Highest Product Offerings

In [0]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- crawled_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- out_of_stock: boolean (nullable = true)
 |-- pid: string (nullable = true)
 |-- product_details: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |--  : string (nullable = true)
 |    |    |-- Alteration Required: string (nullable = true)
 |    |    |-- Animal Source: string (nullable = true)
 |    |    |-- Belt Loops: string (nullable = true)
 |    |    |-- Bottom Fabric: string (nullable = true)
 |    |    |-- Bottom Length: string (nullable = true)
 |    |    |-- Bottom Type: string (nullable = true)
 |    |    |-- Brand: string (nulla

In [0]:
from pyspark.sql.functions import col

In [0]:
filter_df =df.filter(col("brand").isNotNull() & (col("brand") != ""))

In [0]:
filter_df.limit(10).show()

+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
|                 _id|actual_price|average_rating|brand|            category|          crawled_at|         description|discount|              images|out_of_stock|             pid|     product_details|           seller|selling_price|sub_category|               title|                 url|
+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
|fa8e22d6-c0b6-522...|       2,999|           3.9| York|Clothing and Acce...|02/10/2021, 20:11:51|Yorker trackpants...| 69% off|[https:/

In [0]:
# calculate the total number of brands 
brand_count = filter_df.groupBy("brand").count()
brand_count.show()

+--------------------+-----+
|               brand|count|
+--------------------+-----+
|                   K|   46|
|            T10 Spor|  285|
|                 fas|    5|
|                MILD|    9|
|            ECKO Unl|  993|
|    ertugrul ghazi c|   26|
|SATDEVANGIKHADIBHAND|  200|
|veerutess landing...|   14|
|          Marca Disa|  444|
|            Uber Urb|  223|
|              Aeload|    4|
|              SOXCRA|    1|
|          fashion mu|    2|
|            A J STYL|   91|
|      Jagdish Garmen|  157|
|               LIVST|   22|
|           Modest Ci|    1|
|             FITLETH|    1|
|                ALMO|   15|
|                  Pu|  996|
+--------------------+-----+
only showing top 20 rows



In [0]:
# Top 10 brands with the most products
brand_df = brand_count.sort(col("count").desc()).limit(10)
brand_df.show()

+------------+-----+
|       brand|count|
+------------+-----+
|        ARBO|  999|
|          Pu|  996|
|        REEB|  996|
|     True Bl|  996|
|    ECKO Unl|  993|
|Free Authori|  864|
|         Keo|  668|
|         Amp|  621|
|  Black Beat|  560|
|        PixF|  528|
+------------+-----+



In [0]:
brand_df.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/brand_count")

In [0]:
dbutils.fs.ls("/mnt/datalake/delta/brand_count")

[FileInfo(path='dbfs:/mnt/datalake/delta/brand_count/_delta_log/', name='_delta_log/', size=0, modificationTime=1697997388000),
 FileInfo(path='dbfs:/mnt/datalake/delta/brand_count/part-00000-0dfd9eea-02a9-49b0-a514-295309e63511.c000.snappy.parquet', name='part-00000-0dfd9eea-02a9-49b0-a514-295309e63511.c000.snappy.parquet', size=891, modificationTime=1697997393000)]

In [0]:
df_delta = spark.read.format("delta").load("/mnt/datalake/delta/brand_count").show()

+------------+-----+
|       brand|count|
+------------+-----+
|        ARBO|  999|
|          Pu|  996|
|        REEB|  996|
|     True Bl|  996|
|    ECKO Unl|  993|
|Free Authori|  864|
|         Keo|  668|
|         Amp|  621|
|  Black Beat|  560|
|        PixF|  528|
+------------+-----+



## Average Product Rating Across Categories

In [0]:
# calculate average category rating (CategoryAverageRating)
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- crawled_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- out_of_stock: boolean (nullable = true)
 |-- pid: string (nullable = true)
 |-- product_details: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |--  : string (nullable = true)
 |    |    |-- Alteration Required: string (nullable = true)
 |    |    |-- Animal Source: string (nullable = true)
 |    |    |-- Belt Loops: string (nullable = true)
 |    |    |-- Bottom Fabric: string (nullable = true)
 |    |    |-- Bottom Length: string (nullable = true)
 |    |    |-- Bottom Type: string (nullable = true)
 |    |    |-- Brand: string (nulla

In [0]:
average_category_rating = df.select("category", "average_rating")
average_category_rating.show()

+--------------------+--------------+
|            category|average_rating|
+--------------------+--------------+
|Clothing and Acce...|           3.9|
|Clothing and Acce...|           3.9|
|Clothing and Acce...|           3.9|
|Clothing and Acce...|           3.9|
|Clothing and Acce...|           3.9|
|Clothing and Acce...|           3.8|
|Clothing and Acce...|           3.8|
|Clothing and Acce...|           3.8|
|Clothing and Acce...|           3.8|
|Clothing and Acce...|           3.7|
|Clothing and Acce...|           3.8|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.3|
|Clothing and Acce...|           3.9|
+--------------------+--------------+
only showing top 20 rows



In [0]:
# convert the ratings column to float
average_category_rating_df = average_category_rating.withColumn("average_rating", col("average_rating").cast("float"))

In [0]:
average_category_rating_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- average_rating: float (nullable = true)



In [0]:
from pyspark.sql.functions import col, avg, round
result_df = average_category_rating_df.groupby("category").agg(round(avg("average_rating"), 2).alias("average_rating"))

In [0]:
result_df.show()

+--------------------+--------------+
|            category|average_rating|
+--------------------+--------------+
|            Footwear|          3.88|
|Bags, Wallets & B...|          4.13|
|Clothing and Acce...|          3.63|
|                Toys|           3.6|
+--------------------+--------------+



In [0]:
category_delta=result_df.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/category_avg_rating")

In [0]:
spark.read.format("delta").load("/mnt/datalake/delta/category_avg_rating").show()

+--------------------+--------------+
|            category|average_rating|
+--------------------+--------------+
|            Footwear|          3.88|
|Bags, Wallets & B...|          4.13|
|Clothing and Acce...|          3.63|
|                Toys|           3.6|
+--------------------+--------------+



In [0]:
%fs ls /mnt/datalake/delta

path,name,size,modificationTime
dbfs:/mnt/datalake/delta/brand_count/,brand_count/,0,1697997388000
dbfs:/mnt/datalake/delta/category_avg_rating/,category_avg_rating/,0,1697999208000


## Total Count of Products Utilizing specific Discounts

In [0]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- crawled_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- out_of_stock: boolean (nullable = true)
 |-- pid: string (nullable = true)
 |-- product_details: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |--  : string (nullable = true)
 |    |    |-- Alteration Required: string (nullable = true)
 |    |    |-- Animal Source: string (nullable = true)
 |    |    |-- Belt Loops: string (nullable = true)
 |    |    |-- Bottom Fabric: string (nullable = true)
 |    |    |-- Bottom Length: string (nullable = true)
 |    |    |-- Bottom Type: string (nullable = true)
 |    |    |-- Brand: string (nulla

In [0]:
df_filter_id = df.filter(col("_id").isNotNull() & (col("_id") != ""))
df_filter_id.show()


+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
|                 _id|actual_price|average_rating|brand|            category|          crawled_at|         description|discount|              images|out_of_stock|             pid|     product_details|           seller|selling_price|sub_category|               title|                 url|
+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
|fa8e22d6-c0b6-522...|       2,999|           3.9| York|Clothing and Acce...|02/10/2021, 20:11:51|Yorker trackpants...| 69% off|[https:/

In [0]:
df_filter_id.createOrReplaceTempView("temp_table")

In [0]:
%scala
import org.apache.spark.sql.functions._

val df = spark.sql("select * from temp_table")

val result_discount_df = df.withColumn("percentage", regexp_extract(col("discount"), "(\\d+)%", 1).cast("int"))
.filter(col("percentage").isNotNull)
.groupBy("percentage")
.agg(count("_id").alias("count"))
.sort(col("count").desc)

result_discount_df.show()

+----------+-----+
percentage|count|
+----------+-----+
 60| 2335|
 50| 2164|
 35| 1154|
 63| 1017|
 57| 888|
 44| 817|
 40| 805|
 55| 804|
 66| 795|
 64| 787|
 65| 768|
 45| 736|
 52| 710|
 30| 701|
 61| 683|
 76| 594|
 62| 555|
 53| 516|
 58| 490|
 20| 490|
+----------+-----+
only showing top 20 rows

import org.apache.spark.sql.functions._
df: org.apache.spark.sql.DataFrame = [_id: string, actual_price: string ... 15 more fields]
result_discount_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [percentage: int, count: bigint]

In [0]:
%scala
result_discount_df.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/discount_count")

In [0]:
spark.read.format("delta").load("/mnt/datalake/delta/discount_count").show()

+----------+-----+
|percentage|count|
+----------+-----+
|        60| 2335|
|        50| 2164|
|        35| 1154|
|        63| 1017|
|        57|  888|
|        44|  817|
|        40|  805|
|        55|  804|
|        66|  795|
|        64|  787|
|        65|  768|
|        45|  736|
|        52|  710|
|        30|  701|
|        61|  683|
|        76|  594|
|        62|  555|
|        53|  516|
|        20|  490|
|        58|  490|
+----------+-----+
only showing top 20 rows



## Product Count Across Various Categories

In [0]:
%scala
df.createOrReplaceTempView("my_table")

In [0]:
%scala
import org.apache.spark.sql.functions._
val df = spark.sql("select * from my_table")


import org.apache.spark.sql.functions._
df: org.apache.spark.sql.DataFrame = [_id: string, actual_price: string ... 15 more fields]

In [0]:
%scala
val category_count_df =df.filter(col("_id").isNotNull and (col("_id") =!= ""))
.groupBy("category")
.count()
.sort(col("count").desc)


category_count_df.show()

+--------------------+-----+
 category|count|
+--------------------+-----+
Clothing and Acce...|28971|
 Footwear| 987|
Bags, Wallets & B...| 41|
 Toys| 1|
+--------------------+-----+

category_count_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [category: string, count: bigint]

In [0]:
%scala
category_count_df.write.format("delta").mode("overwrite").save("/mnt/datalake/delta/category_count")


In [0]:
spark.read.format("delta").load("/mnt/datalake/delta/category_count").show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|Clothing and Acce...|28971|
|            Footwear|  987|
|Bags, Wallets & B...|   41|
|                Toys|    1|
+--------------------+-----+



## Top 5 Subcategories with the Largest Product Range

In [0]:
%scala
df.show()

+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
 _id|actual_price|average_rating|brand| category| crawled_at| description|discount| images|out_of_stock| pid| product_details| seller|selling_price|sub_category| title| url|
+--------------------+------------+--------------+-----+--------------------+--------------------+--------------------+--------+--------------------+------------+----------------+--------------------+-----------------+-------------+------------+--------------------+--------------------+
fa8e22d6-c0b6-522...| 2,999| 3.9| York|Clothing and Acce...|02/10/2021, 20:11:51|Yorker trackpants...| 69% off|[https://rukminim...| false|TKPFCZ9EA7H5FYZH|[{NULL, NULL, NUL...|Shyam Enterprises| 921| Bottomwear|Solid Men Multico...|https://www.flipk...|
893e6980-f2a0-531...| 1,499| 3.9| York|Clothing and Acce...|02/10/2021, 20:11:52|Yorker trackpants...| 66% off|[https://rukminim...| false|TKPFCZ9EJZV2UVRZ|[{NULL, NULL, NUL...|Shyam Enterprises| 499| Bottomwear|Solid Men Blue Tr...|https://www.flipk...|
eb4c8eab-8206-59d...| 2,999| 3.9| York|Clothing and Acce...|02/10/2021, 20:11:52|Yorker trackpants...| 68% off|[https://rukminim...| false|TKPFCZ9EHFCY5Z4Y|[{NULL, NULL, NUL...|Shyam Enterprises| 931| Bottomwear|Solid Men Multico...|https://www.flipk...|
3f3f97bb-5faf-57d...| 2,999| 3.9| York|Clothing and Acce...|02/10/2021, 20:11:53|Yorker trackpants...| 69% off|[https://rukminim...| false|TKPFCZ9ESZZ7YWEF|[{NULL, NULL, NUL...|Shyam Enterprises| 911| Bottomwear|Solid Men Multico...|https://www.flipk...|
750caa3d-6264-53c...| 2,999| 3.9| York|Clothing and Acce...|02/10/2021, 20:11:53|Yorker trackpants...| 68% off|[https://rukminim...| false|TKPFCZ9EVXKBSUD7|[{NULL, NULL, NUL...|Shyam Enterprises| 943| Bottomwear|Solid Men Brown, ...|https://www.flipk...|
9af4ac75-230b-5d5...| 4,499| 3.8| York|Clothing and Acce...|02/10/2021, 20:11:54|Yorker trackpants...| 74% off|[https://rukminim...| false|TKPFCZ9EFK9DNWDA|[{NULL, NULL, NUL...|Shyam Enterprises| 1,169| Bottomwear|Solid Men Multico...|https://www.flipk...|
1b1d40cc-4eaa-54c...| 1,499| 3.8| York|Clothing and Acce...|02/10/2021, 20:11:54|Yorker trackpants...| 68% off|[https://rukminim...| false|TKPFDABN3GXYPFHE|[{NULL, NULL, NUL...|Shyam Enterprises| 466| Bottomwear|Solid Men Dark Bl...|https://www.flipk...|
a33cf2e2-c5e4-5f0...| 1,499| 3.8| York|Clothing and Acce...|02/10/2021, 20:11:55|Yorker trackpants...| 63% off|[https://rukminim...| false|TKPFCZ9ESGZYT8NH|[{NULL, NULL, NUL...|Shyam Enterprises| 545| Bottomwear|Solid Men Beige T...|https://www.flipk...|
ab4f0cca-6a84-51e...| 1,499| 3.8| York|Clothing and Acce...|02/10/2021, 20:11:55|Yorker trackpants...| 67% off|[https://rukminim...| false|TKPFCZ9DYU33FFXS|[{NULL, NULL, NUL...|Shyam Enterprises| 489| Bottomwear|Solid Men Black T...|https://www.flipk...|
f8a91e89-c5db-5fa...| 1,499| 3.7| York|Clothing and Acce...|02/10/2021, 20:11:56|Yorker trackpants...| 64% off|[https://rukminim...| false|TKPFDABN4NQFVKZY|[{NULL, NULL, NUL...|Shyam Enterprises| 531| Bottomwear|Solid Men Grey Tr...|https://www.flipk...|
002b976d-e1b3-500...| 4,499| 3.8| York|Clothing and Acce...|02/10/2021, 20:11:56|Yorker trackpants...| 76% off|[https://rukminim...| false|TKPFCZ9ENWGMX23W|[{NULL, NULL, NUL...|Shyam Enterprises| 1,074| Bottomwear|Checkered Men Blu...|https://www.flipk...|
1259d286-33b5-5ba...| 1,800| 3.3| York|Clothing and Acce...|02/10/2021, 20:11:57| | 49% off|[https://rukminim...| false|TKPFZFSHHACG3FHC|[{NULL, NULL, NUL...|Shyam Enterprises| 911| Bottomwear|Solid Men Multico...|https://www.flipk...|
79ed89b0-1e6e-592...| 1,800| 3.3| York|Clothing and Acce...|02/10/2021, 20:11:57| | 49% off|[https://rukminim...| false|TKPFZFSHQPDRGZTM|[{NULL, NULL, NUL...|Shyam Enterprises| 911| Bottomwear|Solid Men Mu

In [0]:
%scala
val subcategory_count_df = df.filter(col("_id").isNotNull and (col("_id") =!= ""))
.groupBy("sub_category")
.count().alias("count")
.sort(col("count").desc)
.limit(5)
subcategory_count_df.show()

+--------------------+-----+
 sub_category|count|
+--------------------+-----+
 Topwear|16575|
 Bottomwear| 3862|
 Winter Wear| 2753|
Innerwear and Swi...| 1774|
Clothing Accessories| 1760|
+--------------------+-----+

subcategory_count_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [sub_category: string, count: bigint]

In [0]:
%scala
subcategory_count_df.write.mode("overwrite").format("delta").save("/mnt/datalake/delta/subcategory_count")

In [0]:
%scala
spark.read.format("delta").load("/mnt/datalake/delta/subcategory_count").show()

+--------------------+-----+
 sub_category|count|
+--------------------+-----+
 Topwear|16575|
 Bottomwear| 3862|
 Winter Wear| 2753|
Innerwear and Swi...| 1774|
Clothing Accessories| 1760|
+--------------------+-----+

## Top 10 Sellers with the Largest Product Offerings

In [0]:
%scala
df.printSchema()

root
-- _id: string (nullable = true)
-- actual_price: string (nullable = true)
-- average_rating: string (nullable = true)
-- brand: string (nullable = true)
-- category: string (nullable = true)
-- crawled_at: string (nullable = true)
-- description: string (nullable = true)
-- discount: string (nullable = true)
-- images: array (nullable = true)
 |-- element: string (containsNull = true)
-- out_of_stock: boolean (nullable = true)
-- pid: string (nullable = true)
-- product_details: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- : string (nullable = true)
 | |-- Alteration Required: string (nullable = true)
 | |-- Animal Source: string (nullable = true)
 | |-- Belt Loops: string (nullable = true)
 | |-- Bottom Fabric: string (nullable = true)
 | |-- Bottom Length: string (nullable = true)
 | |-- Bottom Type: string (nullable = true)
 | |-- Brand: string (nullable = true)
 | |-- Brand Color: string (nullable = true)
 | |-- Brand Fit: string (nullable = true)
 | |-- Bust in inch: string (nullable = true)
 | |-- Care Instructions: string (nullable = true)
 | |-- Care instructions: string (nullable = true)
 | |-- Character: string (nullable = true)
 | |-- Clasp Material: string (nullable = true)
 | |-- Clasp Type: string (nullable = true)
 | |-- Closure: string (nullable = true)
 | |-- Coat Type: string (nullable = true)
 | |-- Collar: string (nullable = true)
 | |-- Color: string (nullable = true)
 | |-- Country of Origin: string (nullable = true)
 | |-- Covered in Warranty: string (nullable = true)
 | |-- Cuff: string (nullable = true)
 | |-- Depth: string (nullable = true)
 | |-- Design: string (nullable = true)
 | |-- Distressed: string (nullable = true)
 | |-- Domestic Warranty: string (nullable = true)
 | |-- Dupatta Length: string (nullable = true)
 | |-- Fabric: string (nullable = true)
 | |-- Fabric Care: string (nullable = true)
 | |-- Fabric Details: string (nullable = true)
 | |-- Fabric care: string (nullable = true)
 | |-- Faded: string (nullable = true)
 | |-- Fastener: string (nullable = true)
 | |-- Fit: string (nullable = true)
 | |-- Fly: string (nullable = true)
 | |-- Foot Coverage: string (nullable = true)
 | |-- Generic Name: string (nullable = true)
 | |-- Heel Pattern: string (nullable = true)
 | |-- Height: string (nullable = true)
 | |-- Hem: string (nullable = true)
 | |-- Hip in inch: string (nullable = true)
 | |-- Hooded: string (nullable = true)
 | |-- Ideal For: string (nullable = true)
 | |-- Inner Material: string (nullable = true)
 | |-- Inseam Length: string (nullable = true)
 | |-- Inside Leg in inch: string (nullable = true)
 | |-- International Warranty: string (nullable = true)
 | |-- Knit Type: string (nullable = true)
 | |-- Leather Type: string (nullable = true)
 | |-- Length: string (nullable = true)
 | |-- Length Type: string (nullable = true)
 | |-- Length in inch: string (nullable = true)
 | |-- Lining: string (nullable = true)
 | |-- Lining Material: string (nullable = true)
 | |-- Material: string (nullable = true)
 | |-- Maximum Age: string (nullable = true)
 | |-- Minimum Age: string (nullable = true)
 | |-- Model Details: string (nullable = true)
 | |-- Model Name: string (nullable = true)
 | |-- Model Number: string (nullable = true)
 | |-- Neck: string (nullable = true)
 | |-- Neck Type: string (nullable = true)
 | |-- Not Covered in Warranty: string (nullable = true)
 | |-- Number of Contents in Sales Package: string (nullable = true)
 | |-- Occasion: string (nullable = true)
 | |-- Other Details: string (nullable = true)
 | |-- Other Dimensions: string (nullable = true)
 | |-- Other Features: string (nullable = true)
 | |-- Outer Material: string (nullable = true)
 | |-- Pack Of: string (nullable = true)
 | |-- Pack of: string (nullable = true)
 | |-- Package contains: string (nullable = true)
 | |-- Pattern: string (nullable = true)
 | |-- Placket: string (nullable = true)
 | |-- Pleated: string (nullable = true)
 | |-- Pleats: string (nullable 

In [0]:
%scala
val seller_count_df = df.filter(col("_id").isNotNull and (col("_id") =!= ""))
.filter(col("seller").isNotNull && length(trim(col("seller"))).gt(0))
.groupBy("seller")
.count().alias("count")
.sort(col("count").desc)
.limit(10)


seller_count_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [seller: string, count: bigint]

In [0]:
%scala
seller_count_df.show()

+--------------------+-----+
 seller|count|
+--------------------+-----+
 RetailNet| 1615|
 ARBOR| 976|
 SandSMarketing| 927|
BioworldMerchandi...| 846|
 Keoti| 668|
 AFFGARMENTS| 623|
 Black Beatle| 560|
 ArvindTrueBlue| 528|
 AMALGUS ENTERPRISE| 492|
Tayab Manch Fashions| 489|
+--------------------+-----+

In [0]:
%scala
seller_count_df.write.mode("overwrite").format("delta").save("/mnt/datalake/delta/seller_count")

In [0]:
%scala
spark.read.format("delta").load("/mnt/datalake/delta/seller_count").show()

+--------------------+-----+
 seller|count|
+--------------------+-----+
 RetailNet| 1615|
 ARBOR| 976|
 SandSMarketing| 927|
BioworldMerchandi...| 846|
 Keoti| 668|
 AFFGARMENTS| 623|
 Black Beatle| 560|
 ArvindTrueBlue| 528|
 AMALGUS ENTERPRISE| 492|
Tayab Manch Fashions| 489|
+--------------------+-----+